<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Постановка-задачи" data-toc-modified-id="Постановка-задачи-1">Постановка задачи</a></span></li><li><span><a href="#Как-будем-решать?" data-toc-modified-id="Как-будем-решать?-2">Как будем решать?</a></span></li><li><span><a href="#Подготовим-данные" data-toc-modified-id="Подготовим-данные-3">Подготовим данные</a></span><ul class="toc-item"><li><span><a href="#Код-из-предедыщих-разделов" data-toc-modified-id="Код-из-предедыщих-разделов-3.1">Код из предедыщих разделов</a></span></li></ul></li><li><span><a href="#Применение-градиентного-спуска" data-toc-modified-id="Применение-градиентного-спуска-4">Применение градиентного спуска</a></span></li></ul></div>

### Постановка задачи

- Пусть мы разрабатываем социальную сеть
- И пусть мы нашли сильную корреляцию между кол-вом друзей у пользователя и кол-вом времени сколько он проводит на сайте
- Мы хотим построить модель описывающую эту связь

### Как будем решать?

Выдвинем гипотезу, что существуют такие константы alpha и beta, что зная кол-во друзей можно сделать предсказание времени:

In [3]:
def predict(alpha: float, beta: float, x_i: float) -> float: 
    return beta * x_i + alpha

Ошибку предсказаний будем считать как сумму квадратов отклонений:

In [4]:
from typing import List
Vector = List[float]

def error(alpha: float, beta: float, x_i: float, y_i: float) -> float:
    return predict(alpha, beta, x_i) - y_i

def sum_of_sqerrors(alpha: float, beta: float, x: Vector, y: Vector) -> float: 
    return sum(error(alpha, beta, x_i, y_i) ** 2
               for x_i, y_i in zip(x, y))

Решение будет состоит в том что выбрать такие alpha и beta, чтобы квадрат ошибок был минимальным:
- Это можно сделать аналитически, но в общем случае это будет слишком долго по вычислениям
- Поэтому применим градиентный спуск

### Подготовим данные

In [11]:
num_friends = [i for i in range(-100, 110, 10)] 
daily_minutes = [0.9 * i + 23 for i in num_friends]

#### Код из предедыщих разделов

In [15]:
from typing import List
import math

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v: Vector) -> float:
    return dot(v, v)

def subtract(v: Vector, w: Vector) -> Vector:
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def magnitude(v: Vector) -> float:
    return math.sqrt(sum_of_squares(v))

def add(v: Vector, w: Vector) -> Vector:
    return [v_i + w_i for v_i, w_i in zip(v,w)]

def scalar_multiply(c: float, v: Vector) -> Vector:
    return [c * v_i for v_i in v]

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector: 
    """Двигаемся с шагом `step_size` в направлении `gradient` от `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

### Применение градиентного спуска

In [14]:
import random
import tqdm

num_epochs = 10000
random.seed(0)
guess = [random.random(), random.random()] # choose random value to start

learning_rate = 0.00001

with tqdm.trange(num_epochs) as t: 
    for _ in t:
        alpha, beta = guess
        
        # Partial derivative of loss with respect to alpha
        grad_a = sum(2 * error(alpha, beta, x_i, y_i)
                     for x_i, y_i in zip(num_friends,
                                         daily_minutes))
        
        # Partial derivative of loss with respect to beta
        grad_b = sum(2 * error(alpha, beta, x_i, y_i) * x_i 
                     for x_i, y_i in zip(num_friends,
                                         daily_minutes))
        
        # Compute loss to stick in the tqdm description
        loss = sum_of_sqerrors(alpha, beta,
                               num_friends, daily_minutes)
        
        t.set_description(f"loss: {loss:.3f}") 
        
        # Finally, update the guess
        guess = gradient_step(guess, [grad_a, grad_b], -learning_rate) 

# We should get pretty much the same results:
alpha, beta = guess 
print(f"{alpha=} {beta=}")
assert 22.5 < alpha < 23.0 
assert 0.85 < beta < 0.95

loss: 841.592:  29%|██████▏              | 2923/10000 [00:02<00:04, 1493.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

loss: 25.367:  71%|███████████████▋      | 7108/10000 [00:04<00:01, 1630.14it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

loss: 2.316: 100%|██████████████████████| 10000/10000 [00:06<00:00, 1481.23it/s]

alpha=22.66805731055039 beta=0.9
